### Generación de frases con MCMC

In [11]:
import numpy as np
from collections import defaultdict

In [15]:
alphabet = 'abcdefghijklmnñopqrstuvwxyz áéíóú'

def clean(phrase):
    cleaned_phrase = ''
    for i in phrase:
        if i in alphabet:
            cleaned_phrase += i
    return cleaned_phrase

with open('AC8/corpus.txt', encoding='ISO 8859-1') as f:
    phrases = [clean(i.strip().lower()) for i in f.readlines()]

Para hacer más robusto el método puse todas las palabras en minúscula, para que dos ocurrencias de la misma palabra no tengan distintas posibilidades de continuación dependiendo de si estaban en mayúscula o no. Este método simple tiene limitaciones por ejemplo en cuanto a no respetar nombres propios, etc.

Lo que se hizo es simplemente permitir únicamente las letras del abecedario y los tildes.

In [40]:
contexts = defaultdict(list)
for phrase in phrases:
    words = phrase.split(' ')
    for i in range(len(words)):
        if i + 1 < len(words):
            following = words[i + 1]
            if following:  # avoid adding '' as a word
                contexts[words[i]].append(following)

Genero los contextos aplicando lo sugerido, teniendo cuidado de los bordes. Uso _defaultdict_ para poder realizar _append_ sin tener que pasar por el paso innecesario de tener que inicializar con la lista vacía.

In [41]:
words = [item for sublist in [i.split(' ') for i in phrases] for item in sublist if item] # flat

Genero una lista de todas las palabras posibles para seleccionar la _seed_.

In [35]:
lengths = np.random.randint(20, 40, size=20)
results = []
for length in lengths:
    seed = np.random.choice(words)
    phrase = seed
    previous = seed
    for i in range(length):
        posibilities = contexts[previous]
        if not posibilities:
            break
        chosen = np.random.choice(posibilities)
        phrase += f' {chosen}'
        previous = chosen
    results.append(phrase)

Escojo largos aleatorios de frase para generar, y genero aleatoriamente una semilla para cada una, luego obtengo la palabra siguiente con el método propuesto. Tengo cuidado de no intentar elegir una palabra si el contexto de una palabra es vacío (cosa que sucede con las palabras finales de una frase).

In [42]:
for i in range(len(results)):
    print(f'{i + 1}) {results[i]}\n')

1) problemática y el tiempo mínimo de educación señala el ministerio los países fue una actitud pasiva del sur y sus actividades individuales no se encuentra en marzo de chile colombia paraguay y los créditos o simplemente el

2) se tenían que merma la segunda ronda del recinto comercial se utilizaba para el diferencial entre otros durante los us del recinto comercial se puede hablar de barcelona a día sostuvo la población de formación

3) adultos y perú tomaron esa determinación de robar fernet cerveza y chile comenzó aportando solo se hace difícil poder estudiar y sembrado número uno del total

4) se utilizaba para poder rendir académicamente sumado a nivel país aportó con el porcentaje pero después rafa se encuentra en américa del

5) partido fue creado en la salud mental de chile viven una situación lamentable y con preocupación cómo la protesta que si se extendió dicho encuentro por la estabilidad y sus distintos programas de las horas y chile que saldrá

6) créditos o privadas en

Se podría argumentar que un supuesto fuerte es la utilización de la distribución uniforme para generar las siguientes palabras a partir del contexto, sin embargo, no es tan así dado que como se arman los contextos, una palabra puede estar repetida en el contexto, y eso modifica implícitamente la distribución, por lo que cada palabra no tiene la misma probabilidad de ser escogida, sino que esa probabilidad se ajusta por la cantidad de apariciones

Por otro lado, un supuesto grande es que el contexto únicamente mira un paso hacia adelante y no es capaz de mirar más allá. Esto se aprecia en las frases dado que mirando pares de palabras, es coherente, pero ya la frase entera no tanto.

Lo anterior nos lleva al supuesto más grande, que es que la generación de palabras depende fuertemente de las palabras a las que antecede cada una. Esto no siempre es real, y puede tener mucho que ver por ejemplo con las palabras anteriores, ya sea una o varias, y también con las siguientes, no necesariamente 1.

En los comentarios anteriores se vislumbran varias mejoras posibles, mejorar los contextos, ampliarlos, modificar el algoritmo para que pondere quizás un paso por algún porcentaje y dos pasos por otro (1 paso me refiero a palabra siguiente, 2 a la subsiguiente, etc). Los contextos podrían tener frases chicas en vez de palabras, se podría ponderar el pasado de la frase, etc.

Una mejora evidente pero no menor, es la de aumentar el _corpus_, ya que a mayor cantidad mejor va a ser "el conocimiento" del lenguaje, esto es, la distribución se va a ir ajustando mejor a como funciona realmente la probabilidad de escoger la siguiente palabra, pero claro, asumiendo que la generación de frases tiene que ver con una palabra y no con más cosas que solo 1 paso al futuro.